### Работа с базой данных через Django ORM

Django ORM (Object-Relational Mapping) – это мощный инструмент, который позволяет взаимодействовать с базой данных через Python-код, без необходимости писать SQL-запросы вручную. Django ORM преобразует запросы, выполненные через Python-классы и методы, в SQL и обратно, обеспечивая высокоуровневый интерфейс для работы с данными.


---

## Основные понятия

1. **Модель** – это Python-класс, который описывает таблицу базы данных. Каждое поле модели – это колонка таблицы, а каждый объект модели – это строка в таблице.
2. **QuerySet** – это набор данных, извлеченных из базы данных, которые могут быть отфильтрованы, отсортированы и изменены.
3. **Менеджер модели** – это объект, предоставляющий интерфейс для выполнения операций с данными в базе, таких как создание, обновление, удаление и выборка объектов.

---

## 1. Создание моделей

Модель создается как подкласс `django.db.models.Model`. Каждое поле модели соответствует полю в базе данных, и Django автоматически создаст таблицу для этой модели при выполнении миграций.

```python
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)
    email = models.EmailField()

class Post(models.Model):
    title = models.CharField(max_length=200)
    content = models.TextField()
    published_date = models.DateTimeField()
    author = models.ForeignKey(Author, on_delete=models.CASCADE)
```

### Поля модели

Каждое поле в модели – это объект класса `Field`, который определяет тип данных и правила его обработки. Django поддерживает множество типов полей:

- `CharField` – строка с фиксированной длиной.
- `TextField` – большое текстовое поле.
- `IntegerField` – целое число.
- `DateTimeField` – дата и время.
- `ForeignKey` – внешняя связь (связь "многие-к-одному").

### Аргументы полей

- `max_length` – максимальная длина для строковых полей.
- `null=True` – разрешает хранить в поле значение `NULL`.
- `blank=True` – разрешает оставлять поле пустым при заполнении формы.
- `default` – задает значение по умолчанию для поля.
- `on_delete` – поведение при удалении связанных объектов (например, `CASCADE` – удаление всех связанных объектов).

---

## 2. Миграции

Django использует систему миграций для управления изменениями в структуре базы данных.

- **Создание миграций**:
  После создания или изменения модели нужно создать и применить миграцию, чтобы изменения отразились в базе данных.

  ```bash
  python manage.py makemigrations
  python manage.py migrate
  ```

- **Автоматическое создание таблиц**:
  Django создаст соответствующие таблицы и связи в базе данных на основе ваших моделей.

---

## 3. CRUD операции (Create, Read, Update, Delete)

### 3.1. Создание объектов (Create)

Чтобы создать объект в базе данных, достаточно вызвать метод `.save()` или использовать метод `.create()` менеджера модели.

#### Пример 1: Использование `.save()`
```python
author = Author(name='John Doe', email='john@example.com')
author.save()  # Сохраняет объект в базу данных
```

#### Пример 2: Использование `.create()`
```python
Post.objects.create(title='My first post', content='Hello, world!', author=author)
```

### 3.2. Чтение данных (Read)

Для выборки данных используется менеджер модели, который по умолчанию доступен через `objects`. Он возвращает объекты типа `QuerySet`, который поддерживает фильтрацию, сортировку и агрегацию данных.

#### Пример 1: Получить все объекты модели
```python
all_posts = Post.objects.all()
```

#### Пример 2: Фильтрация объектов
Фильтрация выполняется с помощью метода `.filter()`. Можно передавать ключевые аргументы для указания условий поиска.

```python
published_posts = Post.objects.filter(published_date__isnull=False)
```

#### Пример 3: Получение одного объекта
Для получения одного объекта используют метод `.get()`, который выбрасывает ошибку `DoesNotExist`, если объект не найден.

```python
post = Post.objects.get(id=1)
```

### 3.3. Обновление данных (Update)

Объект можно обновить, изменив его поля и сохранив изменения с помощью метода `.save()`. Также можно массово обновлять объекты с помощью метода `.update()`.

#### Пример 1: Обновление одного объекта
```python
post = Post.objects.get(id=1)
post.title = 'Updated title'
post.save()
```

#### Пример 2: Массовое обновление объектов
```python
Post.objects.filter(published_date__isnull=True).update(status='draft')
```

### 3.4. Удаление объектов (Delete)

Объекты можно удалить с помощью метода `.delete()`. Удалять можно как отдельные объекты, так и наборы данных.

#### Пример 1: Удаление одного объекта
```python
post = Post.objects.get(id=1)
post.delete()
```

#### Пример 2: Массовое удаление
```python
Post.objects.filter(status='draft').delete()
```

---

## 4. Фильтрация, сортировка и агрегация

### 4.1. Фильтрация

Django ORM поддерживает сложные запросы с использованием различных фильтров.

- **Полный или частичный поиск**:
  - `__exact` – точное совпадение.
  - `__icontains` – частичное совпадение без учета регистра.
  - `__startswith` – начинается с.

  ```python
  Post.objects.filter(title__icontains='django')
  ```

- **Фильтрация по диапазону**:
  - `__range` – поиск значений в диапазоне.
  - `__gte`, `__lte` – больше или равно, меньше или равно.

  ```python
  Post.objects.filter(published_date__range=['2023-01-01', '2023-12-31'])
  ```

- **Работа с датами**:
  - `__year`, `__month`, `__day` – фильтрация по годам, месяцам или дням.

  ```python
  Post.objects.filter(published_date__year=2023)
  ```

### 4.2. Сортировка

Для сортировки результатов используется метод `.order_by()`.

- `order_by('field')` – сортировка по возрастанию.
- `order_by('-field')` – сортировка по убыванию.

```python
Post.objects.order_by('published_date')
```

### 4.3. Ограничение количества записей

Метод `.count()` позволяет быстро узнать количество объектов, соответствующих условиям.

```python
Post.objects.filter(status='published').count()
```

### 4.4. Аггрегация данных

Django ORM поддерживает различные агрегации, такие как подсчет, сумма, среднее и т.д., через методы агрегации из `django.db.models`.

```python
from django.db.models import Count, Avg

# Подсчет количества постов каждого автора
author_post_count = Author.objects.annotate(num_posts=Count('post'))

# Средняя длина контента постов
average_content_length = Post.objects.aggregate(Avg('content__length'))
```

---

## 5. Связи между моделями

Django ORM поддерживает следующие типы связей между моделями:

- **One-to-Many (ForeignKey)**:
  Связь "один ко многим", которая выражает отношение, где один объект может быть связан с множеством других.

  ```python
  class Post(models.Model):
      author = models.ForeignKey(Author, on_delete=models.CASCADE)
  ```

- **Many-to-Many (ManyToManyField)**:
  Связь "многие ко многим", которая используется, когда одна запись может быть связана с множеством других, и наоборот.

  ```python
  class Post(models.Model):
      tags = models.ManyToManyField(Tag)
  ```

---

## 6. Транзакции

Django ORM поддерживает транзакции через контекстный менеджер `transaction.atomic()`. Это позволяет выполнить несколько операций с базой данных как одну транзакцию, которая либо полностью выполняется, либо откатывается.

```python
from django.db import transaction

with transaction.atomic():
    post = Post.objects.create(title='New Post', content='Post content')
    comment = Comment.objects.create(post=post, content='First comment')
```

---

Работа с базой данных через Django ORM упрощает взаимодействие с данными, избавляя разработчика от необходимости писать SQL-запросы. Django ORM предоставляет интуитивный интерфейс для создания, чтения, обновления и удаления данных, а также для выполнения сложных запросов с фильтрацией, сортировкой и агрегацией данных.